In [1]:
%load_ext autoreload
%autoreload 2 # autoreload all modules

In [2]:
%matplotlib inline
import sys
import numpy as np
import sciunit

from sciunit.models.examples import ConstModel # model always predicts a fixed const
from sciunit.capabilities import ProducesNumber

sys.path.append("../")

Importing self-defined module and functions

In [3]:
from src.ldmunit.models.poisson import PoissonModel
from src.ldmunit.scores import MSEScore
from src.ldmunit.scores import NLLScore
from src.ldmunit.capabilities import ProducesLoglikelihood

## Stimulate datasets

In [4]:
observation_0 = {'value': np.random.randint(1, 100, size = (100,))} # random integers
observation_1 = {'value': np.random.poisson(9, size = (100,))} # Poisson with lambda = 9
observation_2 = {'value': np.random.poisson(10, size = (100,))}
observation_3 = {'value': np.random.poisson(11, size = (100,))}

## Import the models classes for comparsion

In [5]:
poi_10 = PoissonModel(10, name="Poisson 10")
const_37 = ConstModel(37, name="Constant 37")

## Create test class

Return the mean-sqaured-error (MSE) from a model of `ProducesNumber` capability

In [6]:
class MSETest(sciunit.Test):
    """Returns the MSE score for each model"""   
    
    required_capabilities = (ProducesNumber,) # The one capability required for a model to take this test.  
    score_type = MSEScore # This test's 'judge' method will return a BooleanScore.  
    
    def validate_observation(self, observation):
        if type(observation) is not dict:
            raise ObservationError("Observation must be a python dictionary")
        if 'value' not in observation:
            raise ObservationError("Observation must contain a 'value' entry")
        
    def generate_prediction(self, model):
        """Generate predictions
        """
        return model.produce_number()
    
    def compute_score(self, observation, prediction):
        score = MSEScore.compute(observation, prediction)
        score.description = ("A MSE score")
        return score

In [7]:
mse_test_0 = MSETest(observation_0, name='random integers from 1:100')
mse_test_0.judge(const_37)

MSE = 8.854

## Test data/matrix

When tested againest certain model, each test instance equipped with a `judge` function with returns the correspounding score type (Ratio, Boolean, Float, ...)

Multiple test instances can be initialized as such. `TestSuite` class also provides a wrapper-like functionality in multiple test instances.

Similarly, `judge` can be called on `TestSuite` instances for multiple testing against a __single__ model, which returns a score matrix

In [8]:
mse_test_1 = MSETest(observation_1, name='lambda: 9, poisson')
mse_test_2 = MSETest(observation_2, name='lambda: 10, poisson')
mse_test_3 = MSETest(observation_3, name='lambda: 11, poisson')


mse_test_suite = sciunit.TestSuite([mse_test_0, mse_test_1, mse_test_2, mse_test_3], name="MSE test suite")
mse_test_suite.judge(const_37)

,random integers from 1:100,"lambda: 9, poisson","lambda: 10, poisson","lambda: 11, poisson"
Constant 37,MSE = 8.854,MSE = 7.895,MSE = 7.370,MSE = 6.829


### A single test aginst different models
Analogously, a __single__ test instance or a test suite can be tested across different model.

In [9]:
const_9 = ConstModel(9, name="Constant 9")
const_10 = ConstModel(10, name="Constant 10")

mse_test_suite.judge([const_37, const_9, const_10])

,random integers from 1:100,"lambda: 9, poisson","lambda: 10, poisson","lambda: 11, poisson"
Constant 37,MSE = 8.854,MSE = 7.895,MSE = 7.370,MSE = 6.829
Constant 9,MSE = 22.059,MSE = 0.089,MSE = 0.101,MSE = 0.148
Constant 10,MSE = 21.317,MSE = 0.097,MSE = 0.091,MSE = 0.116


## Test for negative log-likelihood

A model of `ProducesLoglikelihood` is required

In [10]:
class NLLTest(sciunit.Test):

    required_capabilities = (ProducesLoglikelihood, )
    score_type = NLLScore
 
    def generate_prediction(self, model):
        return model.produce_loglikelihood()
    
    def compute_score(self, observation, prediction):
        score = NLLScore.compute(observation, prediction)
        return score

In [11]:
from src.ldmunit.models.poisson import PoissonAltModel
pos_10_alt = PoissonAltModel(10, name="Poisson 10 (alt)")

In [12]:
log_poi_test_0 = NLLTest(observation_0, name='random integers from 1:100')
log_poi_test_1 = NLLTest(observation_1, name='lambda: 9, poisson')
log_poi_test_2 = NLLTest(observation_2, name='lambda: 10, poisson')
log_poi_test_3 = NLLTest(observation_3, name='lambda: 11, poisson')

In [13]:
log_poi_test_2.judge(pos_10_alt)

A negative log likelihood = -1307.190

In [14]:
log_test_suite = sciunit.TestSuite([log_poi_test_0, log_poi_test_1, log_poi_test_2, log_poi_test_3])
log_test_suite.judge(pos_10_alt)

,random integers from 1:100,"lambda: 9, poisson","lambda: 10, poisson","lambda: 11, poisson"
Poisson 10 (alt),A negative log likelihood = -9725.441,A negative log likelihood = -1086.142,A negative log likelihood = -1307.190,A negative log likelihood = -1548.962
